In [1]:
%pip install boto3
# Libraries maybe you need to install in your environment to use selenium and chromedriver below
%pip install selenium chromedriver-binary

   ---------------------------------------- 0.0/139.3 kB ? eta -:--:--
   -------- ------------------------------- 30.7/139.3 kB ? eta -:--:--
   -------------------------------- ------- 112.6/139.3 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 139.3/139.3 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/12.0 MB ? eta -:--:--
   - -------------------------------------- 0.4/12.0 MB 5.9 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/12.0 MB 5.8 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/12.0 MB 6.1 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/12.0 MB 8.2 MB/s eta 0:00:02
   -------- ------------------------------- 2.7/12.0 MB 8.9 MB/s eta 0:00:02
   ------------ --------------------------- 3.8/12.0 MB 11.0 MB/s eta 0:00:01
   ---------------- ----------------------- 4.9/12.0 MB 12.6 MB/s eta 0:00:01
   -------------


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for chromedriver-binary: filename=chromedriver_binary-125.0.6379.3.0-py3-none-any.whl size=8609877 sha256=6cafb9cbe3584eb809c5d5123f276263d9bb4b62d9d976e0ca77b51ed9a6a81a
  Stored in directory: c:\users\20520\appdata\local\pip\cache\wheels\c6\f1\44\6c7a24a5242b0f0ae29cfec52f476b5f7988b7db4223589bb4
Successfully built chromedriver-binary
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#import necessary libraries

import os
import boto3
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

In [51]:
# Set the download directory
download_directory = os.getcwd()

# Set the desired download options
prefs = {
    "download.default_directory": download_directory,
}

options = Options()
options.add_experimental_option("prefs", prefs)

# Initialize the WebDriver
driver = webdriver.Chrome(options=options)

# Open the Google Drive URL
url_link = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC"
driver.get(url_link)

# Wait for 2 seconds
time.sleep(2)

# Click on the folder to access its contents
xpath_folder = '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[2]/div'
folder = driver.find_element(By.XPATH, xpath_folder)
folder.click()

# Click on the download button
xpath_download = '//*[@id=":1"]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[6]/div/span'
download = driver.find_element(By.XPATH, xpath_download)
download.click()

# Wait for 5 seconds
time.sleep(5)

# Quit the WebDriver
driver.quit()

In [52]:
import zipfile
import os

# Extract the contents of the "data.zip" file
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [58]:
# Define S3 storage
obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'ZfDdneDqDMWO07RvroVX')
obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'v8V87ClJJwLAIqBPll1ix4V93Vgjdpbt0sowjt5k')
obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')


In [59]:
# Function to upload file to minio
def upload_file_to_minio(file_path, minio_bucket, minio_object_name):
    s3c = boto3.resource('s3',
                        endpoint_url=obj_storage_endpoint,
                        aws_access_key_id=obj_storage_access_key,
                        aws_secret_access_key=obj_storage_secret_key,
                        config=boto3.session.Config(signature_version='s3v4'),
                        verify=False
                        )
    s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [60]:
# Upload file to minio
upload_file_to_minio('data.json', 'data', 'data-raw/data.json')
upload_file_to_minio('data2.json', 'data', 'data-raw/data2.json')
upload_file_to_minio('data3.json', 'data', 'data-raw/data3.json')